# Open Street Map Scrape, Cleaning and SQL DB Import

## Exploratory Analysis - Philadelphia, PA Open Street Map (OSM) XML

In [3]:
# Sample the 500MB XML file to see what data fields are available and how the data are structured between node, way and relation elements. 
import xml.etree.cElementTree as ET

# Sample 10% of file
sample_file_k10 = "sample10.osm"

# Sample %20
sample_file_k5 = "sample5.osm"

# Sample %30
sample_file_k3 = "sample3.osm"

# whole file
osm_file = "philadelphia_pennsylvania.osm"

# set paramater to take every kth top-level element
# k = 10
k = 5
# k=3

# [LXML](http://lxml.de) is definitely faster to read and summarize the OSM XML tree than BeautifulSoup, though 
# editing and rewriting the tree seemed simpler with bs4. Since the full file is too large to process locally, we'll
# sample it to create two test OSM XML files: one with %20 of the elements and one with 33%. 

def get_element(osm_file, tags=('node', 'way', 'relation')):
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags: # Yield only node, way and relation tags
            yield elem
            root.clear()

# Write every kth 'node', 'way' and 'relation' element to the test file
with open(sample_file_k5, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ') 
    
    for i, elem in enumerate(get_element(osm_file)):
        if i % k == 0: # When i divided by k leaves remainder 0
            output.write(ET.tostring(elem, encoding='utf-8'))
    
    output.write('</osm>')

In [4]:
# Count each of the root node children's elements. Takes a little over 1 minute to iterate through the full OSM XML file.

from collections import Counter

def count_level_I_tags(filename):
#         import time
#         start_time = time.time()
        root = ET.parse(filename).getroot()
        items = []
        for child in root.iter():
            tag_name = child.tag
            items.append(tag_name)
        tags = Counter(items)
        return tags
#         print("--- {}min ---".format((time.time() - start_time)/60))

In [ ]:
# There are over 2.8 million node tags, over 261 thousand way tags and only about 4000 relation tags. There are also 
# over 1.7 million tags nested below node, way and relation elements.
# http://wiki.openstreetmap.org/wiki/OSM_XML

# This would make a good chart - top ten tags in the OSM file v. bottom 10

# count_level_I_tags('sample10.osm')
# count_level_I_tags('sample5.osm')
# count_level_I_tags('sample3.osm')
count_level_I_tags('philadelphia_pennsylvania.osm')

In [46]:
# All tags seem to be in order, none floating around where they shouldn't be. 
def count_orphans(filename):
        root = ET.parse(filename).getroot()
        items = []
        for node in root.findall('tag'):
            tag_name = node.get('k')
            items.append(tag_name)
        orphan_tags = Counter(items)
#         return orphan_tags
        print len(orphan_tags)

In [47]:
count_orphans(osm_file)

0


In [16]:
from bs4 import BeautifulSoup

# Have a glance at what the street tags look like - 3466 tags have 'addr:street' attributes
soup = BeautifulSoup(open('sample5.osm', "r+b"), "xml")
    
street_tags = soup.find_all("tag", attrs={"k": "addr:street"})

len(street_tags)

3466

In [17]:
# Even though we found no orphan tags under the root, there may be a few orphan tags with addr:street attributes nested further 
# down in the XML tree
parents = []

for tag in street_tags:
    par_n = tag.find_parent("node")
    par_w = tag.find_parent("way")
    parents.append(par_n)
    parents.append(par_w)

parent_list_nan = filter(lambda x: x==None, parents)

rr = len(parents)
tt = len(parent_list_nan)
tot = rr - tt

table = {'Tags_Nparent_yes' : rr, 'Tags_Nparent_no' : tt, 'Total_Node_tags': tot}

print table

{'Total_Node_tags': 3457, 'Tags_Nparent_yes': 6932, 'Tags_Nparent_no': 3475}


In [50]:
# Find all unique tag names amongst the children tags of parent tags 'node', 'way', and 'relation'.
# Takes a little over 5 seconds to run over sample_file_k5, about 20% of the dataset. 
# Takes over 37 seconds on local 2.8 GHz Intel Core i7 with 16 GB flash memory to run over the full OSM file.

import pprint

def count_tags(filename):
        import time
        start_time = time.time()
        root = ET.parse(filename).getroot()
        items = []
        for node in root.findall('node/tag'):
            tag_name = node.get('k')
            items.append(tag_name)
        for way in root.findall('way/tag'):
            tag_name = way.get('k')
            items.append(tag_name)
        for rel in root.findall('relation/tag'):
            tag_name = rel.get('k')
            items.append(tag_name)
        all_tags = Counter(items)
        return dict(all_tags)
#         print len(items)
#         print len(all_tags)
#         print("--- {} seconds ---".format((time.time() - start_time)))

In [51]:
# Tags with more records will make better sample queries after we port over to SQL. Others are mystifiying. 'PA:ANALGROUP': 1, 'wood', 'surveillance', 'supervised': 43
# 'shelter': 11, 'power': 872, 'parking': 252. We have 854 unique tags, many with few entries (percentage under 4 entries?)
# SELECT count(), tag FROM table WHERE count(tag) < 5
# Suggests a more consistent tagging procedure would greatly improve the map's metadata. 
# Is ingesting map data from a Google Maps API easier? No. 
# way and relation tags seem sparser than node tags. 
# There are 1720896 tags, so none are orphans unattached to a node, way or relation, the OSM XML schema's three 
# data primitives

# Could also scrape http://wiki.openstreetmap.org/wiki/Map_Features to ensure that only recognized tag key-value pairs
# can be entered in the dataset as tag elements. At least would be interesting to compare the list below with the list
# scraped from OSM sight. With luck, the same fuzzy string matching technique will work on both, and we can clean up the 
# tags a bit. 

# Node.'amenity': 1607
# 'addr:street': 1500
# 'traffic_calming': 38
# 'social_facility': 61,
# 'public_transport': 46
# 'website': 153,

# Looks like Tiger has also listed out name bases with more than one word.  If we cleanse and merge these lists, could
# we improve our street name matching approach? 

#          'tiger:name_base_1': 1550,
#          'tiger:name_base_2': 171,
#          'tiger:name_base_3': 8,

# count_tags(sample_file_k10)
# count_tags(sample_file_k5)
count_tags(osm_file)

{'health_facility:type': 3,
 'tiger:source': 9477,
 'maxspeed': 4755,
 'gnis:import_uuid': 1154,
 'alt_business': 1,
 'turn:lanes:forward': 1051,
 'massgis:MGMT_ZIP': 4,
 'is_in': 1904,
 'source:old_ref': 169,
 'addr:state': 7554,
 'start_date:source': 1,
 'name:am': 1,
 'playground': 2,
 'fax': 38,
 'attribution': 16027,
 'Keyword': 1,
 'source:nat_name:date': 3,
 'dispense': 3,
 'icao': 6,
 'parking:condition:time_interval': 1,
 'motor_vehicle': 681,
 'NHD:Phone': 1,
 'faa': 2,
 'condition': 1,
 'addr:street': 17179,
 'alt_name_2': 1,
 'source:name': 594,
 'school': 1,
 'alt_name_1': 1,
 'ref': 11922,
 'image:2': 1,
 'is_in:continent': 2,
 'name:map-bms': 1,
 'name:ilo': 1,
 'service:bicycle:repair': 2,
 'disused': 43,
 'source:footway': 1,
 'massgis:MGMT': 4,
 'name:lanes': 1,
 'name:lez': 1,
 'gnis:state_id': 3410,
 'bicycle': 4039,
 'Price and Price Elder Law': 1,
 'roof:height': 38,
 'Max_PDOP': 50,
 'MUNICIPAL_': 1,
 'bridge:name': 1,
 'name:lbe': 1,
 'piste:type': 15,
 'leaf_ty

## Build Canonical List of Street Names for Fuzzy String Matching

### Two lists of street names from Philly Open Gov weren't cleaned or comprehensive


In [3]:
# create expected list from scraped name strings at http://www.geographic.org/streetview/usa/pa/philadelphia.html
# how does it compare to the tiger name attributes in the way objects? Compare all these sets of names? 
#The first philly gov dataset turns out not to be very useful. 
import pandas as pd

philly_streets_canon = pd.read_csv('http://gis.phila.gov/gisdata/ODP/STR_AliasList.csv')

philly_streets_canon.describe()


/usr/local/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,OBJECTID,SEG_ID,SUF_DIR
count,16045.00000,1.604500e+04,0.0
mean,8023.00000,4.670222e+05,NaN
std,4631.93687,1.834571e+05,NaN
min,1.00000,1.000010e+05,NaN
25%,4012.00000,3.403090e+05,NaN
50%,8023.00000,4.222040e+05,NaN
75%,12034.00000,5.423380e+05,NaN
max,16045.00000,1.160162e+06,NaN


In [2]:
philly_streets_canon.head()

,OBJECTID,SEG_ID,PRE_DIR,NAME,TYPE,SUF_DIR
0,346,100199.0,S,EIGHTY FOURTH,ST,NaN
1,347,100199.0,S,EIGHTY-FOURTH,ST,NaN
2,348,100200.0,S,EIGHTY FOURTH,ST,NaN
3,349,100200.0,S,EIGHTY-FOURTH,ST,NaN
4,350,100206.0,NaN,RECIRCULATION,RD,NaN


In [4]:
# Coerce variables to string - some Nan values are typed as integers and don't play nicely
philly_streets_canon[['NAME', 'TYPE']] = philly_streets_canon[['NAME', 'TYPE']].astype(str)
# philly_streets_canon = philly_streets_canon[[philly_streets_canon.NAME.notnull(), philly_streets_canon.TYPE.notnull()]]

philly_streets_canon['Type'] = philly_streets_canon['TYPE'].apply(lambda x: x.lower().capitalize())

philly_streets_canon['Type'] = philly_streets_canon['Type'].replace(philly_streets_canon.Type.values, 
                                            [mapping.get(word, word) for word in philly_streets_canon.Type.values])

# philly_streets_canon['full_name'] = philly_streets_canon['full_name'].replace(philly_streets_canon.full_name.values, 
#                                       [mapping.get(word, word) for word in philly_streets_canon.full_name.values])

# Some missing values need to be taken out
# philly_streets_canon['TYPE'][9]


NameError: name 'mapping' is not defined

In [9]:
philly_streets_canon['full_name'] = philly_streets_canon[['NAME', 'Type']].apply(lambda x: ' '.join(x), axis=1)

philly_streets_canon['full_name'] = philly_streets_canon['full_name'].apply(lambda x: x.capitalize())

philly_streets_canon['full_name'].unique()

array(['Eighty fourth street', 'Eighty-fourth street',
       'Recirculation road', 'I 95 nan', 'I95 nan', 'I 95 expressway',
       'I95 expressway', '291 highway', 'Pa 291 highway',
       'Route 291 highway', 'State highway 291 highway', 'I 95 ramp',
       'I95 ramp', '95 ramp nan', '95 rmp nan', '291 nan', 'I291 nan',
       'Route 291 nan', 'Seventy eighth street', 'Seventy-eighth street',
       'Sixty sixth street', 'Sixty-sixth street', 'Sixty seventh street',
       'Sixty-seventh street', 'Sixty fifth street', 'Sixty-fifth street',
       'Sixty third street', 'Sixty-third street', 'Sixty fourth street',
       'Sixty-fourth street', 'Sixty eighth street', 'Sixty-eighth street',
       'Sixty nineth street', 'Sixty-nineth street', 'Seventieth street',
       'Seventy first street', 'Seventy-first street',
       'Seventy second street', 'Seventy-second street',
       'Eightieth street', 'Seventy seventh street',
       'Seventy-seventh street', 'Seventy fourth street',
    

In [97]:
# Mispellings here are also not great:

# ((\b[tfsen].+y\b)-? ?(\b[tfsen].+[thrdstnd]\b) street)
# ([tfsen].+y) ?-?(?P<tens>[tfsen].+[th|rd|st|nd])|(?P=tens) street

#Replace with a range of numbers paired with 'Street' 1-90, 64-79 weird little avenues right at the edge of Cheltenham
numbered_streets = re.compile(r'(\b[\dtzfsen].+[thndo]\b street)', re.IGNORECASE)

philly_streets_canon['full_name'] = philly_streets_canon['full_name'].replace(numbered_streets,'')

philly_streets_canon['full_name'].unique()

array(['', 'Recirculation road', 'I 95 nan', 'I95 nan', 'I 95 expressway',
       'I95 expressway', '291 highway', 'Pa 291 highway',
       'Route 291 highway', 'State highway 291 highway', 'I 95 ramp',
       'I95 ramp', '95 ramp nan', '95 rmp nan', '291 nan', 'I291 nan',
       'Route 291 nan', 'I 76 nan', 'I 76 street', 'I76 nan', 'I76 street',
       'Muhlfeld street', 'Mcclellan street', 'St marks street',
       'St albans street', 'Vodges street', 'Cobbs creek parkway',
       'St michael drive', 'St christopher drive', 'Columbus boulevard',
       'Chris columbus boulevard', 'Delaware avenue', 'Concourse drive',
       'Access road', 'Mckean street', 'Waverly wlk nan',
       'Lawrence court wlk nan', 'St peters way', 'Lombard mew nan',
       'Service drive', 'Fifty fifth drive', 'Fifty-fifth drive',
       'Fifty fifth terrace', 'Fifty-fifth terrace',
       'Surgeon general court', 'Woodland wlk nan', 'Grays ferry trail',
       'University avenue', 'Addison wlk nan', 'End d

In [111]:
philly_street_type_canon = list(philly_streets_canon['Type'].unique())

# philly_street_type_canon = philly_street_type_canon.remove('Nan')

print philly_street_type_canon

['Street', 'Road', 'Nan', 'Expressway', 'Highway', 'Ramp', 'Parkway', 'Drive', 'Boulevard', 'Avenue', 'Way', 'Terrace', 'Court', 'Trail', 'Square', 'Place', 'Exit', 'Tunnel', 'Row', 'Lane', 'Walk', 'Crescent', 'Circle']


In [13]:
# Let's try another way by getting all the unique names from a list of placenames in Philly. Still no guarantee it's 
# comprehensive, even though the file is provided by Philadelphia.gov's open data project. https://www.opendataphilly.org/dataset/street-place-names/resource/ed10ab8b-e998-4853-b440-81df22c39bfc
psc = pd.read_csv('http://gis.phila.gov/gisdata/ODP/STR_PlaceNames.csv')
psc.head()

,OBJECTID,ALIAS,ADDRESS
0,219,Allen M Sterne School,1655 UNITY ST
1,220,Allens La Art Center,601 W ALLENS LN
2,221,Allens Lane Park,200 NIPPON ST
3,222,Allens Lane Sta,200 NIPPON ST
4,223,Allens Lane Station,200 NIPPON ST


In [73]:
# A list like this would be good to restrict form input data for Open Street Map. 

psc['ADDRESS'] = psc['ADDRESS'].astype(str)

psc['Address'] = psc['ADDRESS'].replace(re.compile(r'^\d+(?!th|rd|st|nd)'), '').apply(lambda x: x.lower())

streets = list(psc['Address'].unique())

for idx, s in enumerate(streets):
    words = s.split()
    words = [x.capitalize() for x in words]
    clean = ' '.join(str(mapping.get(word, word)) for word in words)
    streets[idx] = clean

print streets

['Unity Street', 'West Allens Lane', 'Nippon Street', 'Cecil B Moore Avenue', 'Walnut Street', 'Erdrick Street', 'South Broad Street', 'Locust Walk', 'North Independence Ml West', 'North Broad Street', 'Red Lion Road', 'Chestnut Street', 'North 15th Street', 'Pine Street', 'South 16th Street', 'Locust Street', 'Pattison Avenue', 'North Independence Ml West', 'South 48th Street', 'South 23rd Street', 'Longshore Avenue', 'Cresco Avenue', 'North 17th Street', 'Market Street', 'East Ontario Street and Richmond Street', 'East Washington Lane', 'Richmond Street', 'Germantown Avenue', 'South 33rd Street', 'South 60th Street', 'Cobbs Creek Parkway', 'Cobbs Creek Parkway and South 61st Street', 'Ridge Avenue', 'Cathedral Road', 'East Mount Airy Avenue', 'Lindbergh Boulevard', 'Prospect Avenue', 'Bowler Street', 'North 13th Street', 'Walnut Street', 'North 42nd Street and Parkside Avenue', 'Arch Street', 'Bainbridge Street', 'South 22nd Street', 'East Roosevelt Boulevard', 'Aramingo Avenue', 'Bu

In [70]:
# This seems too low, think this file is only for streets that may have multiple aliases. 
len(streets)

1213

### Fortunately, the US Census TIGER data includes street names

In [ ]:
## Though the addr:street attributes in the node tags contain errors, the ways, imported from the US Census' TIGER 
# dataset, are cleaner. We can use these to build a canonical reference list of street name strings. 
<way changeset="37014829" id="43117631" timestamp="2016-02-05T08:40:51Z" uid="3057995" user="oini" version="11">
  <nd ref="110421617"/>
  <nd ref="2906080683"/>
  <tag k="name" v="West Girard Avenue"/>
  <tag k="layer" v="1"/>
  <tag k="bridge" v="yes"/>
  <tag k="highway" v="secondary"/>
  <tag k="tiger:cfcc" v="A21"/>
  <tag k="ref:penndot" v="SR2008"/>
  <tag k="tiger:zip_left" v="19131"/>
  <tag k="tiger:name_base" v="Girard"/>
  <tag k="tiger:name_type" v="Ave"/>
  <tag k="tiger:zip_right" v="19104"/>
  <tag k="tiger:name_base_1" v="United States Highway 30"/>
  <tag k="old_ref_legislative" v="67301"/>
  <tag k="tiger:name_direction_prefix" v="W"/>
 </way>

In [ ]:
# Sometimes a 'name' attribute is the street name, other times it's the name of a location or establishment 
<way changeset="10677130" id="150255661" timestamp="2012-02-13T20:23:59Z" uid="594684" user="kumpel75" version="1">
  <nd ref="1631799891"/>
  <nd ref="1631799894"/>
  <nd ref="1631799920"/>
  <nd ref="1631799922"/>
  <nd ref="1631799962"/>
  <nd ref="1631799959"/>
  <nd ref="1631799945"/>
  <nd ref="1631799914"/>
  <nd ref="1631799891"/>
  <tag k="name" v="Red Lion Diner"/>
  <tag k="building" v="restaurant"/>
  <tag k="addr:street" v="US and US Streets"/>
  <tag k="addr:postcode" v="08088"/>
  <tag k="addr:housename" v="Red Lion Diner"/>
 </way>

In [8]:
from bs4 import BeautifulSoup
#Let's compare how many unique street names we have in the osm file

street_soup = BeautifulSoup(open('philadelphia_pennsylvania.osm', "r+b"), "xml")

street_ns = []
# dirs = []
# types = []

#Could compare the address with the name in the way tag, or just replace it? Need to get tiger name bases for 1 and 2 
#word bases 
street_names_tig = street_soup.find_all("tag", attrs={"k": "tiger:name_base"})
for i in street_names_tig:
    street_ns.append(i['v'])

# street_directions = street_soup.find_all("tag", attrs={"k": "tiger:name_direction_prefix"})
# for i in street_directions:
#     dirs.append(i['v'])

# type_types = street_soup.find_all("tag", attrs={"k": "tiger:name_type"})
# for i in type_types:
#     types.append(i['v'])



In [27]:
# Collect all the name attributes in the tags nested under way elements. 
import xml.etree.cElementTree as ET

waynames = []

root = ET.parse(osm_file).getroot()

def is_street(tag):
    return (tag.attrib['k'] == "name")

for node in root.findall('way/tag'):
    if is_street(node):
        waynames.append(node.attrib['v'])
            
# street_way_names = street_soup.find_all("tag", attrs={"k": "name"})
# for i in street_way_names:
#     waynames.append(i['v'])
    
d = {'full_name': waynames}

In [28]:
# s is a list of the base names, and w is a list of full street name strings, with any house numbers stripped from the beginning.

import pandas as pd
# s = pd.Series(d['base_name']).unique()
# t = pd.Series(d['type']).unique()
numbers = re.compile(r'^\d+(?!th|rd|st|nd)')
w = list(pd.Series(d['full_name']).str.replace('^\d+(?!th|rd|st|nd)', '').unique())
w
# w = w.unique()

['Seaport Drive',
 'Industrial Highway',
 'Morton Avenue',
 'Bullens Lane',
 'Chester Road',
 '9th Street',
 'Chestnut Street',
 'Schuylkill River',
 'Pennsylvania Turnpike',
 'Valley Avenue',
 'North 1st Road',
 'South Liberty Street',
 'White Horse Pike',
 'North Street',
 'Linda Avenue',
 'Batsto Fireline Road',
 'Virginia Avenue',
 'North Packard Street',
 'Tomocomo Drive',
 'North Union Road',
 '3rd Street',
 'Nesco Road',
 'Ranere Avenue',
 'Main Road',
 'Grape Street',
 'Sleeper Branch Road',
 'Pine Road',
 'Middle Road',
 'Walnut Street',
 'Anderson Avenue',
 'Columbia Road',
 'Berwyn Avenue',
 'Plymouth Place Drive',
 'Colwell Avenue',
 'West 15th Street',
 'Rizzotte Drive',
 'East 15th Street',
 'Michael Lane',
 'West Lakeview Drive',
 'Mount Carmel Lane',
 'Phillip Avenue',
 'South Myrtle Street',
 'North Myrtle Street',
 'North Egg Harbor Road',
 'Nescochague Road',
 'Broadway',
 'Rail Way',
 'West Mills Road',
 'Wayne Avenue',
 'Neal Road',
 'South Lakeview Drive',
 'Hange

In [10]:
len(s)

15284

In [11]:
# Now we at least have a list of all the basenames, of which there are 15284.  We don't have the type endings or 
# directional prefixes, though.  At least we know how many different street names there are. Might also be good to 
#pull the highways names from here. u'Mantua;Harrison' some strange items with semi-colons

# u'Pederson' What's up with this? There are some clearly problematic items on the list, though they may or may not 
# cause a problem for approximate string matching. Also appear to be clustered at the end of the list.

# u'Norfolk Southern Railway:Pennsylvania Railroad', u'Belmont;Green', u'United States Highway 1; Lincoln', 
# u'Mantua;Harrison', u'Reading Railroad:Septa Railroad', u'Baltimore and Ohio Railroad:Norfolk Southern Railway', 
# u'Franklin:Hampton', u'State Route 68; State Route 68; State Route 68A; State Route 68; State Route 68', 
# u'United States Highway 206;Old York', u'Perry; Lincoln', u'Norfolk Southern Railway; Csx Railway; Conrail Railroad', 
# u'Township Line;Big Oak', u'Market:United States Highway 13 (Bus)', u'Coursey; College', u'I-295:I-76', u'Early; Davis',
# u'of the Arts', u'\x7f\x7fBeech', u'Bridge; Main', Spring:Pond View', u'Cypress:Longacre', u'Delmar;84th', 
# u'New Jersey Transit:Penn Central Railroad; Conrail Railroad'

philly_street_name_canon = list(s)
print list(s)

[u'Valley', u'1st', u'Liberty', u'White Horse', u'North', u'Linda', u'Batsto Fireline', u'Virginia', u'Packard', u'Tomocomo', u'Union', u'3rd', u'Nesco', u'Ranere', u'Main', u'Grape', u'Sleeper Branch', u'Pine', u'Middle', u'Walnut', u'Anderson', u'Columbia', u'Berwyn', u'Colwell', u'15th', u'Rizzotte', u'Michael', u'Lakeview', u'Mount Carmel', u'Phillip', u'Myrtle', u'Egg Harbor', u'Nescochague', u'Broadway', u'Rail', u'West Mills', u'Wayne', u'Neal', u'Hanger', u'Sindoni', u'Pleasant', u'Neals', u'Airport', u'Sycamore', u'Johnson', u'Atsion Batsto', u'Old Forks', u'Elvins', u'Mardor', u'Laurel', u'Basin', u'Hankins', u'Woodlawn', u'Girard', u'Vine', u'2nd', u'Mazza', u'Amanda', u'Atlantic', u'Pleasant Mills', u'Academy', u'Gatto', u'D Daves', u'Taylor', u'Rose Rita', u'Wells', u'French', u'Trenton', u'Pratt', u'Francis', u'Brynmawr', u'Marlyn', u'Richards', u'Sandy Causeway', u'Bellevue', u'Aimee Jean', u'Oak', u'Central Railroad of New Jersey:Conrail Railroad', u'14th', u'Chews', u'

In [11]:
len(philly_street_name_canon)

15284

In [120]:
# Unfortunately, the Tiger name bases may not be needed if the full name strings work well enough. We'll write the base names to disk
# just in case they're needed another time.
import csv

with open('./philly_street_base_names_canon.txt', 'wb') as basesphilly:
    for item in unicode(philly_street_name_canon):
        basesphilly.write(item)

In [74]:
# Let's have a look at the full name strings to see if this list of full name strings can be used as a canonical reference for 
# fuzzy string matching
type(w)

numpy.ndarray

In [80]:
len(w)

40943

In [29]:
#Also needs to be cleaned with mapping for Rd, Dr
print list(w)

['Seaport Drive', 'Industrial Highway', 'Morton Avenue', 'Bullens Lane', 'Chester Road', '9th Street', 'Chestnut Street', 'Schuylkill River', 'Pennsylvania Turnpike', 'Valley Avenue', 'North 1st Road', 'South Liberty Street', 'White Horse Pike', 'North Street', 'Linda Avenue', 'Batsto Fireline Road', 'Virginia Avenue', 'North Packard Street', 'Tomocomo Drive', 'North Union Road', '3rd Street', 'Nesco Road', 'Ranere Avenue', 'Main Road', 'Grape Street', 'Sleeper Branch Road', 'Pine Road', 'Middle Road', 'Walnut Street', 'Anderson Avenue', 'Columbia Road', 'Berwyn Avenue', 'Plymouth Place Drive', 'Colwell Avenue', 'West 15th Street', 'Rizzotte Drive', 'East 15th Street', 'Michael Lane', 'West Lakeview Drive', 'Mount Carmel Lane', 'Phillip Avenue', 'South Myrtle Street', 'North Myrtle Street', 'North Egg Harbor Road', 'Nescochague Road', 'Broadway', 'Rail Way', 'West Mills Road', 'Wayne Avenue', 'Neal Road', 'South Lakeview Drive', 'Hanger Road', 'North 3rd Street', 'Sindoni Lane', 'Pleas

In [45]:
expected

['Street',
 'Streets',
 'Avenue',
 'Boulevard',
 'Drive',
 'Court',
 'Place',
 'Square',
 'Lane',
 'Road',
 'Bypass',
 'Trail',
 'Parkway',
 'Commons',
 'Pike',
 'Alley',
 'Circle',
 'East',
 'North',
 'South',
 'West',
 'Crossing',
 'Extension',
 'Highway',
 'Plaza',
 'Terrace',
 'Walk',
 'Way',
 'Run',
 'Tunnel',
 'Broadway',
 'Park',
 'Close']

In [47]:
# Clean out the 'name' tags that refer to placenames rather than street names by filtering with regex to ensure that all strings
# terminate with an expected ending. 
street_names_list = w

street_yes = re.compile(r'(\bstreets?|avenue|boulevard|drive|court|place|square|lane|road|trail|parkway|commons|pike|alley|circle|east|north|south|west|extension|highway|plaza|terrace|walk|run|tunnel|broadway|close|park|crossing|bypass\b)$', re.IGNORECASE)

clean_street_name_canon = []

for idx, i in enumerate(street_names_list):
    match = street_yes.search(i)
    if match: 
        clean_street_name_canon.append(i)
        

In [48]:
len(clean_street_name_canon)

31489

In [46]:
# 'Pepperoncini': 1, ? What is going on with even the Tiger names?! Over 2000 different endings. 
from collections import Counter
endings = []

for street in clean_street_name_canon:
    x = street.split(' ')
#     y = len(x)
    endings.append(x[-1])

Counter(endings)
    
# len(list(pd.Series(endings).unique()))

Counter({'Alley': 364,
         'Avenue': 4315,
         'Boardwalk': 1,
         'Boulevard': 459,
         'Broad': 1,
         'Broadway': 7,
         'Bypass': 11,
         'Center/Park': 1,
         'Circle': 1251,
         'Close': 9,
         'Commons': 12,
         'Court': 3289,
         'Crossing': 22,
         'Crosswalk': 1,
         'Ddrive': 1,
         'Drive': 4785,
         'East': 114,
         'Expresspark': 1,
         'Extension': 10,
         'GoshenTrail': 1,
         'Group-West': 1,
         'Highway': 32,
         'Homeplace': 1,
         'Jeshurun': 2,
         'Lane': 3902,
         'Marketplace': 1,
         'North': 97,
         'Northeast': 2,
         'PArkway': 1,
         'Park': 453,
         'Parkway': 66,
         'Pike': 94,
         'Place': 1170,
         'Plaza': 74,
         'ROad': 1,
         'Railroad': 1,
         'Riverwalk': 3,
         'Road': 5978,
         'Run': 196,
         'Sidewalk': 3,
         'Skatepark': 1,
         'South': 9

In [79]:
#15000 basenames with 31489 total combinations of directional prefixes and road type suffixes makes sense. 
#Using this as a list of choices, we can employ approximate (fuzzy) string matching to test if street name strings
# match list items.  We can adjust matches based on Levenstein Distance, which calculates how many characters 
# strings share as a numerical matching score. The same logic could be used via Django to reject input to the addr:name
#attribute

# Using the larger Census TIGER National Road dataset could work for n-gram matching, though it would need to be hosted on S3 and 
# processing would need to be done via Spark on AWS EMR or Databricks. 

clean_street_name_canon

['Seaport Drive',
 'Industrial Highway',
 'Morton Avenue',
 'Bullens Lane',
 'Chester Road',
 '9th Street',
 'Chestnut Street',
 'Pennsylvania Turnpike',
 'Valley Avenue',
 'North 1st Road',
 'South Liberty Street',
 'White Horse Pike',
 'North Street',
 'Linda Avenue',
 'Batsto Fireline Road',
 'Virginia Avenue',
 'North Packard Street',
 'Tomocomo Drive',
 'North Union Road',
 '3rd Street',
 'Nesco Road',
 'Ranere Avenue',
 'Main Road',
 'Grape Street',
 'Sleeper Branch Road',
 'Pine Road',
 'Middle Road',
 'Walnut Street',
 'Anderson Avenue',
 'Columbia Road',
 'Berwyn Avenue',
 'Plymouth Place Drive',
 'Colwell Avenue',
 'West 15th Street',
 'Rizzotte Drive',
 'East 15th Street',
 'Michael Lane',
 'West Lakeview Drive',
 'Mount Carmel Lane',
 'Phillip Avenue',
 'South Myrtle Street',
 'North Myrtle Street',
 'North Egg Harbor Road',
 'Nescochague Road',
 'Broadway',
 'West Mills Road',
 'Wayne Avenue',
 'Neal Road',
 'South Lakeview Drive',
 'Hanger Road',
 'North 3rd Street',
 'Si

In [49]:
# We can use this as a list of choices for fuzzy string matching on the full names in the addr:street attributes of tags
import csv

with open('./philly_street_full_names_canon.csv', 'w') as streetsphilly:
    wr = csv.writer(streetsphilly, quoting=csv.QUOTE_ALL)
    wr.writerow(clean_street_name_canon)

## Cleaning OSM File

### Create Audit Function to Review Street Names in OSM Files

In [50]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint
import csv

# Matches words beginning with any non-whitespace character that repeats >1 time, possibly ends with a period and 
# occurs at the end of a line. 
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE) 

# Search through a list of strings to confirm they terminate with items in the 'expected' list.
# If strings end with an item that's not a member of 'expected,' add the group of strings surrounding this item to the
# list 'street_types'

expected = ["Street", "Streets", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", "Bypass",
            "Trail", "Parkway", "Commons", "Pike", "Alley", "Circle", "East", "North", "South", "West", "Crossing",
            "Extension", "Highway", "Plaza", "Terrace", "Walk", "Way", "Run", "Tunnel", "Broadway", "Park", "Close"]

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

# Audit an osm file and return a dictionary of strings that don't terminate with any of the words in the expected list 
def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

In [73]:
audit(sample_file_k5)

defaultdict(set,
            {'206': {'US 70 & US 206'},
             '33': {'Route 33'},
             '37th': {'N 37th'},
             '43rd': {'N 43rd'},
             '446-1234': {'1 Brookline BlvdHavertown, PA 19083(610) 446-1234'},
             '5': {'West Girard Avenue, 5'},
             '70': {'NJ 70', 'US 70'},
             '73': {'New Jersey 73'},
             '80': {'N Lewis RD Unit #80'},
             'Ave': {'Aramingo Ave',
              'Cottman Ave',
              'Devon St & Mt. Pleasant Ave',
              'E. Mt Airy Ave',
              'Fairmount Ave',
              'Fort Washington Ave',
              'Frankford Ave',
              'Germantown Ave',
              'Grays Ave',
              'Hirst Ave',
              'Montgomery Ave',
              'Park Ave',
              'Parkway Ave',
              'S Clinton Ave',
              'Stenton Ave',
              'West Girard Ave'},
             'Ave.': {'Bonny Brook Ave.',
              'East Butler Ave.',
             

In [98]:
audit(osm_file)

defaultdict(set,
            {'1': {'Route 1'},
             '111': {'South Clinton Avenue Ste. 111'},
             '13': {'W Main St #13'},
             '168': {'Marlton Pike East Ste. 168'},
             '19047': {'200 Manor Ave. Langhorne, PA 19047',
              '2245 E. Lincoln Hwy, Langhorne, PA 19047',
              '2275 E Lincoln Hwy, Langhorne, PA 19047',
              '2300  East Lincoln Highway, Pennsylvania 19047'},
             '19067': {'East Trenton Avenue Morrisville, PA 19067'},
             '206': {'US 206', 'US 70 & US 206'},
             '33': {'Route 33'},
             '37th': {'N 37th'},
             '38': {'New Jersey 38', 'New Jersey Route 38', 'Route 38'},
             '39th': {'N 39th'},
             '40': {'1140 US Highway 40', 'Rt 40'},
             '4080': {'4080'},
             '41st': {'S. 41st'},
             '43rd': {'N 43rd'},
             '446-1234': {'1 Brookline BlvdHavertown, PA 19083(610) 446-1234'},
             '452': {'Market Street; Pennsylv

### Test Code for Processing Functions

In [23]:
test_osm_xml = """
<?xml version="1.0" encoding="UTF-8"?>
<osm>
 <node changeset="34353963" id="1483624883" lat="39.9787384" lon="-75.3038692" timestamp="2015-09-30T19:10:51Z" uid="3276050" user="JCarden" version="2">
  <tag k="name" v="Kettle"/>
  <tag k="amenity" v="restaurant"/>
  <tag k="cuisine" v="Diner"/>
  <tag k="addr:street" v="1 Brookline Boulevard Havertown PA 19083(610) 446-1234"/>
  <tag k="addr:postcode" v="19083"/>
  <tag k="addr:housenumber" v="1"/>
 </node>
</osm>
"""

# Retrieve all tags with the "addr:street" attribute
test_soup = BeautifulSoup(test_osm_xml, "xml")
    
street_tgs = test_soup.find_all("tag", attrs={"k": "addr:street"})

# Iterate through the list of these tags searching for strings that match regex patterns for other fields, like telephone numbers.
# If any strings belong in other fields, add a new tag with the correct attribute. 
n_tags = []

for idx, tag in enumerate(street_tgs):
    call_me = phone.search(tag['v'])
    if call_me:
        v_val = call_me.group()
        new_tag = test_soup.new_tag("tag", k="phone", v='{}'.format(v_val))
        n_tags.append(new_tag)
        tag.insert_after(new_tag)
        
test_soup
# n_tags
# type(n_tags[0])

# Retrieve the parent node of a given tag
for tag in street_tgs:
    par = tag.find_parent("node")
    ss = par.find_all("tag", attrs={"k": "full"})

# ss
par


<node changeset="34353963" id="1483624883" lat="39.9787384" lon="-75.3038692" timestamp="2015-09-30T19:10:51Z" uid="3276050" user="JCarden" version="2">\n<tag k="name" v="Kettle"/>\n<tag k="amenity" v="restaurant"/>\n<tag k="cuisine" v="Diner"/>\n<tag k="addr:street" v="1 Brookline Boulevard Havertown PA 19083(610) 446-1234"/><tag k="phone" v="(610) 446-1234"/>\n<tag k="addr:postcode" v="19083"/>\n<tag k="addr:housenumber" v="1"/>\n</node>

In [71]:
# After a value for another field is matched, delete it from the orginal string. 
n = "1 Brookline Boulevard Havertown PA 19083(610) 446-1234"
y = pa_state.search(n)
xx = n.split()
# z = y.group()
# type(z)
# z
for idx, l in enumerate(xx):
    keystone = re.search(pa_state, l)
    if keystone:
        ilt = 'Pennsylvania'
        del xx[idx]

ilt
xx

['1', 'Brookline', 'Boulevard', 'Havertown', '19083(610)', '446-1234']

In [92]:
with open('./philly_street_full_names_canon.csv', 'r') as streets_canon:
    reader = csv.reader(streets_canon)
    philly_street_name_canon = list(reader)[0]

# Need to exclude matches under 90. Some chance that errors will be introduced for items with common names, like Spruce
st_name = process.extractOne('pullen al', philly_street_name_canon)
st_name
# if st_name[1] > 90:
        
#   {'206': {'US 70 and US 206'}, ('Campus Crossings E and F street', 86)
#              '33': {'Route 33'}, ('South 33rd Street', 68)
#              '37th': {'North 37th'}, ('North 37th Street', 90)
#              '43rd': {'North 43rd'}, ('North 43rd Street', 90)
#              '446-1234': {'1 Brookline Boulevard Havertown PA 19083(610) 446-1234'}, ('Brookline Boulevard', 90)
#              '5': {'West Girard Avenue, 5'}, ('West Girard Avenue', 95)
#              '70': {'NJ 70', 'US 70'}, ('US 202 Parkway', 86)
#              '73': {'New Jersey 73'}, ('New Road', 86)
#              '80': {'North Lewis Road Unit #80'}, ('Lewis Road', 90)
#              'Bigler': {'Bigler'}, ('Bigler Street', 90)
#              'Chestnut': {'Chestnut'}, ('Chestnut Street', 90)
#              'Mallon': {'Mallon'}, ('Mallon Street', 90) Mallon Avenue returns ('Marlon Avenue', 92)
#              'Moore': {'Cecil B. Moore'}, ('Cecil B Moore Avenue', 95)
#              'NJ-73': {'NJ-73'}, ('NJ 73 South / Atlantic City / Old Marlton Pike', 60)
#              'PA': {'East Lincoln Highway Langhorne PA'}, ('Lincoln Highway', 90)
#              'Spruce': {'Spruce'}, ('Blue Spruce Court', 90)
#              'Vine': {'12th and Vine'}, ('Campus Crossings E and F street', 86)
#              'Warren': {'Warren'}}) ('Warren Street East', 90)
# DAVISVIL LE RO AD, ('Davisville Road', 94)
# pullen al, ('Mullen Road', 70)

('Mullen Road', 70)

In [55]:
test_tag_xml = """
<?xml version="1.0" encoding="UTF-8"?>
<osm>
 <node changeset="34353963" id="1483624883" lat="39.9787384" lon="-75.3038692" timestamp="2015-09-30T19:10:51Z" uid="3276050" user="JCarden" version="2">
  <tag k="name" v="Kettle"/>
  <tag k="amenity" v="restaurant"/>
  <tag k="cuisine" v="Diner"/>
  <tag k="addr:street" v="1 Brookline Boulevard Havertown PA 19083(610) 446-1234"/>
  <tag k="addr:postcode" v="19083"/>
  <tag k="addr:housenumber" v="1"/>
  <tag k="addr:street" v="US 70 and US 206"/>
  <tag k="addr:street" v="Route 33"/>
  <tag k="addr:street" v="West Girard Avenue, 5"/>
  <tag k="addr:street" v="North Lewis Road Unit #80"/>
  <tag k="addr:street" v="Cecil B. Moore"/>
  <tag k="addr:street" v="East Lincoln Highway PA"/>
  <tag k="addr:street" v="12th and Vine"/>
 </node>
</osm>
"""

reload(sys)
sys.setdefaultencoding('utf-8')
# soup = BeautifulSoup(open(sample_file_k5, "r+b"), "xml")
soup = BeautifulSoup(test_tag_xml, "xml")    
# street_tags = soup.find_all("tag", attrs={"k": "addr:street"})
# print soup
street_tags = soup.find_all("tag", attrs={"k" : "addr:street"})
with open('./philly_street_full_names_canon.csv', 'r') as streets_canon:
    reader = csv.reader(streets_canon)
    possible_streets = list(reader)[0]

    no_fix = ['Mallon Avenue']

    for tag in street_tags:
        m = street_type_re.search(tag['v'])
        street_end = m.group()
        intersect = intersection.search(tag['v'])

        if street_end not in expected and not intersect and tag['v'] not in no_fix: 
            st_name = process.extractOne(tag['v'], possible_streets)
            print st_name
            if st_name[1] >= 90:
                tag['v'] = st_name[0]
                print tag['v']

### Main OSM XML Cleaning Functions

In [51]:
# Mapping to replace words may be unnecessary if fuzzy string matching works. 
# Is there way to reverse this logic, so the values become the keys, and any item matched to a value is replaced with
# that value's key? 
mapping = { "St": "Street",
            "St.": "Street",
           "st.": "Street",
           "ST": "Street",
           "st": "Street",
           "Sreet": "Street",
           "Sstreet": "Street",
            "Atreet": "Street",
           "Steet": "Street",
           "street": "Street",
           "Sts.": "Streets",
           "AVE": "Avenue",
           "Ave": "Avenue",
           "Ave.": "Avenue",
           "ave": "Avenue",
           "Av": "Avenue",
           "Ave,": "Avenue",
           "avenue": "Avenue",
           "E": "East",
           "E.": "East",
           "e": "East",
           "N": "North",
           "N.": "North",
           "s": "South",
           "S": "South",
           "S.": "South",
           "south": "South",
           "W": "West",
           "Blvd": "Boulevard", 
           "Blvd.": "Boulevard",
           "Blv": "Boulevard",
           "Blvd,": "Boulevard",
           "Cir": "Circle",
           "Ct": "Court",
           "Dr": "Drive",
           "Ln": "Lane",
           "ln": "Lane",
           "lane": "Lane",
           "Hwy":"Highway",
           "Hwy,": "Highway",
           "PIke": "Pike",
           "Rd": "Road",
           "Rd.": "Road",
           "rd": "Road",
           "road": "Road",
           "ROAD": "Road",
           "RD": "Road",
           "ext": "Extension",
           "way": "Way",
           "&": "and",
           "Exp": "Expressway",
           "Rmp": "Ramp",
           "Pky": "Parkway",
           "Ter": "Terrace",
           "Tr": "Trail",
           "Sq": "Square",
           "Pkwy": "Parkway",
           "pky": "Parkway",
           "Pl": "Place",
           "Ext": "Exit",
           "Wlk": "Walk",
           "wlk": "Walk",
           "Brg": "Bridge",
           "Tun": "Tunnel",
           "Tnl": "Tunnel",
           "Cre": "Crescent",
           "al": "Alley",
           "Ste": "Suite"
            }

In [52]:
import re

# filter out zipcodes, phone numbers, and full addresses and add a tag to the parent node with that attribute if it is
# not already included in the parent node. 

# RE similar to below should be able to detect state and national highways. 

# Between 1 and 4 digits at the beginning or a line, exclude numbered street names like 34th, 23rd, 1st, 42nd
house_number = re.compile(r'^(\d{2,4}|\b\d{1}\b)(?!th|rd|st|nd)', re.IGNORECASE)

# |(?<!route )(?<!us )(?<!nj )(?<!jersey )(\d{2,4}|\b\d{1}\b)(?!th|rd|st|nd)$

# unit_number needs to match two digits at the end of a line, unless they follow a state name Or US or have a number ending
unit_num = re.compile(r'u.*it ?#?(\d{2,4} ?|\b\d{1}\b)(?!th|rd|st|nd)$', re.IGNORECASE)

# Also suite, ste, ste. Little more complicated, as suite has letters in common with "Street"
suite_num = re.compile(r'(?<!u)s[u|t]?[i|t]?t?e?.? ?#?(?<!rsey )(?<!ania )(\d{2,4} ?|\b\d{1}\b)(?!th|rd|st|nd)$', re.IGNORECASE)

 # Five consecutive digits is very likely a zipcode in this context, though could also restrict to a list of known
# zipcodes in the Phillly metro area. 
zipcode = re.compile(r'\d{5}')
# zip_garbage = re.compile(r'\d{5}.{3,5}')

# any chr, 3 digits, any chr, 3 digits, any chr, 4 digits. As bonus, this will ensure that phone numbers in the DB are
# unique
phone = re.compile(r'\(?\d{3}\)?[-\.\s]??\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4}') 

# There should technically be only one city, Philadelphia, though there a few other suburban cities close to Philly included
cities = ['Philadelphia', 'Langhorne', 'Morrisville', 'Havertown', 'Levittown', 'Springfield']

#Try to abbreviate 'Pennsylvania' or 'New Jersey', I dare you! 
pa_state = re.compile(r'\bpen.*\.?ia\b|\bpa\.?\b', re.IGNORECASE)

#Calibrated to accept regional vernacular varients such as 'Joizy' or 'Joisey'
nj_state = re.compile(r'\bnj\b|new ?j.*y|j.*y', re.IGNORECASE)

# If words aren't spaced, like 'BrooklineBlvd'
abutted = re.compile(r'([A-Z]{1}\w+)([A-Z]\w+)')

# ALL CAPS
all_cap = re.compile(r'[A-Z]{3,}') # At least 3 caps 

#fix all lower to normal case
all_low = re.compile(r'\b[a-z]{4,}\b') # At least 4 lower case chrs with whitespace before and after

# If two words joined by 'and' or '&', add "Streets" Doesn't yet match '&amp'
intersection = re.compile(r'(\w+\s\band\b\s\w+)|(\w+\s\b&\b\s\w+)')

In [53]:
import sys
import time
from __future__ import division
from bs4 import BeautifulSoup
import re

# Iterate through the list of words in each street name, check to see if any match a state name, house number, 
# phone number, or zipcode. If one of the words matches, insert it into the tree as a new tag.
# If the OSM file addr:street contains information for other fields, extract this information and create a new tag.

def clean_streets(osmfile):
    start_time = time.time()
    reload(sys)
    sys.setdefaultencoding('utf-8')
    
    soup = BeautifulSoup(open(osmfile, "r+b"), "xml")
    
    street_tags = soup.find_all("tag", attrs={"k": "addr:street"})
    phones = soup.find_all("tag", attrs={"k": "phone"})

    for tag in street_tags:
        par = tag.find_parent("node")
        
        # Check if string contains a housenumber and insert new tag if so
        num = house_number.search(tag['v'])
        if num and par != None:
            snum = par.find_all("tag", attrs={"k": "addr:housenumber"})
            v_val = num.group()
            new_tag = soup.new_tag("tag", k="addr:housenumber", v='{}'.format(v_val))
            if new_tag not in snum:
                tag.insert_after(new_tag)
                re.sub(house_number,'', tag['v']) # None of these substitutions are working
        
        #Check if string contains phone number and insert new tag if so
        call_me = phone.search(tag['v'])
        if call_me:
            v_val = call_me.group()
            new_tag = soup.new_tag("tag", k="phone", v='{}'.format(v_val))
            if new_tag not in phones:
                tag.insert_after(new_tag)
                re.sub(phone,'',tag['v'])
        
        post = zipcode.search(tag['v'])
        if post and par != None:
            ezips = par.find_all("tag", attrs={"k": "addr:postcode"})
            v_val = post.group()
            new_tag = soup.new_tag("tag", k="addr:postcode", v='{}'.format(v_val))
            if new_tag not in ezips:    
                tag.insert_after(new_tag)        
                re.sub(zipcode,'', tag['v'])
                
        sweet = suite_num.search(tag['v'])
        if sweet and par != None: 
            sweets = par.find_all("tag", attrs={"k": "addr:suite"})
            v_val = sweet.group()
            new_tag = soup.new_tag("tag", k="addr:suite", v='{}'.format(v_val))
            if new_tag not in sweets:
                tag.insert_after(new_tag)
                re.sub(suite_num,'', tag['v'])
        
        uno = unit_num.search(tag['v'])
        if uno and par != None: 
            units = par.find_all("tag", attrs={"k": "addr:unit"})
            v_val = uno.group()
            new_tag = soup.new_tag("tag", k="addr:unit", v='{}'.format(v_val))
            if new_tag not in units:
                tag.insert_after(new_tag)
                re.sub(unit_num,'', tag['v'])

        keystone = pa_state.search(tag['v'])
        if keystone:
            new_tag = soup.new_tag("tag", k="addr:state", v="Pennsylvania")
            tag.insert_after(new_tag)
            re.sub(pa_state,'', tag['v'])
            
        joizy = nj_state.search(tag['v'])        
        if joizy:
            new_tag = soup.new_tag("tag", k="addr:city", v="New Jesery")
            tag.insert_after(new_tag)
            re.sub(nj_state,'', tag['v'])
    
    osmfile_write = './full_osm_clean.osm'
    
    with open(osmfile_write, "w") as f:
        f.write(soup.prettify())
    print("--- {}min ---".format((time.time() - start_time)/60))

In [54]:
# 26-28 min runtime
clean_full_osm = './full_osm_clean.osm'
clean_streets(osm_file)

In [55]:
audit(clean_full_osm)

defaultdict(set,
            {'1': {'Route 1'},
             '111': {'South Clinton Avenue Ste. 111'},
             '13': {'W Main St #13'},
             '168': {'Marlton Pike East Ste. 168'},
             '19047': {'200 Manor Ave. Langhorne, PA 19047',
              '2245 E. Lincoln Hwy, Langhorne, PA 19047',
              '2275 E Lincoln Hwy, Langhorne, PA 19047',
              '2300  East Lincoln Highway, Pennsylvania 19047'},
             '19067': {'East Trenton Avenue Morrisville, PA 19067'},
             '206': {'US 206', 'US 70 & US 206'},
             '33': {'Route 33'},
             '37th': {'N 37th'},
             '38': {'New Jersey 38', 'New Jersey Route 38', 'Route 38'},
             '39th': {'N 39th'},
             '40': {'1140 US Highway 40', 'Rt 40'},
             '4080': {'4080'},
             '41st': {'S. 41st'},
             '43rd': {'N 43rd'},
             '446-1234': {'1 Brookline BlvdHavertown, PA 19083(610) 446-1234'},
             '452': {'Market Street; Pennsylv

In [8]:
# 3.5 min runtime on 1/5 file, likely over 17 min for full file. 2nd version taking nearly 6 min for 1/5 file
clean_streets(sample_file_k5)

In [24]:
audit(sample_file_k5)

defaultdict(set,
            {'206': {'US 70 & US 206'},
             '33': {'Route 33'},
             '37th': {'N 37th'},
             '43rd': {'N 43rd'},
             '446-1234': {'1 Brookline BlvdHavertown, PA 19083(610) 446-1234'},
             '5': {'West Girard Avenue, 5'},
             '70': {'NJ 70', 'US 70'},
             '73': {'New Jersey 73'},
             '80': {'N Lewis RD Unit #80'},
             'Ave': {'Aramingo Ave',
              'Cottman Ave',
              'Devon St & Mt. Pleasant Ave',
              'E. Mt Airy Ave',
              'Fairmount Ave',
              'Fort Washington Ave',
              'Frankford Ave',
              'Germantown Ave',
              'Grays Ave',
              'Hirst Ave',
              'Montgomery Ave',
              'Park Ave',
              'Parkway Ave',
              'S Clinton Ave',
              'Stenton Ave',
              'West Girard Ave'},
             'Ave.': {'Bonny Brook Ave.',
              'East Butler Ave.',
             

In [71]:
# Clean up words in the addr:street attributes 
def filter_words(osmfile):
    start_time = time.time()
    reload(sys)
    sys.setdefaultencoding('utf-8')
    soup = BeautifulSoup(open(osmfile, "r+b"), "xml")
    
    street_tags = soup.find_all("tag", attrs={"k": "addr:street"})    

    for tag in street_tags:
        words = tag['v'].split()
            
        street_orphans = ['Spruce', 'Bigler', 'Warren', 'Chippendale', 'Front', 'Greene', 'Market', 'Sloan', 'Salina'
                         'Warren', 'Chestnut', 'North 37th']
        
        if tag['v'] in street_orphans:
            words.append('Street')
            
        av_orphans = ['Mallon']
        
        if tag['v'] in av_orphans:
            words.append('Avenue')
                
        for idx, word in enumerate(words):
            
            if word in cities:
                new_tag = soup.new_tag("tag", k="addr:city", v='{}'.format(word))
                tag.insert_after(new_tag)
                del words[idx]
    
            no_space = re.search(abutted, word)
            if no_space:
                words[idx] = abutted.sub(r'\1', word).strip(',')
                words.insert(idx+1, re.sub(abutted, r'\2', word).strip(','))
            caps = all_cap.search(word) #caps nor lowered don't seem to be working
            if caps:
                words[idx] = word[idx].title()
            lowered = all_low.search(word)
            if lowered:
                words[idx] = words[idx].capitalize()
        
        clean_name = ' '.join(str(mapping.get(word, word)) for word in words).strip(',')
        tag['v'] = str(clean_name)
    
    with open(osmfile, "w") as f:
        f.write(soup.prettify())
    print("--- {}min ---".format((time.time() - start_time)/60))

In [57]:
# 20 min for full file
filter_words(clean_full_osm)

In [58]:
audit(clean_full_osm)

defaultdict(set,
            {'1': {'Route 1'},
             '111': {'South Clinton Avenue Ste. 111'},
             '13': {'West Main Street #13'},
             '168': {'Marlton Pike East Ste. 168'},
             '19047': {'200 Manor Avenue Langhorne, PA 19047',
              '2245 East Lincoln Highway Langhorne, PA 19047',
              '2275 East Lincoln Highway Langhorne, PA 19047',
              '2300 East Lincoln Highway, Pennsylvania 19047'},
             '19067': {'East Trenton Avenue Morrisville, PA 19067'},
             '206': {'US 206', 'US 70 and US 206'},
             '33': {'Route 33'},
             '37th': {'North 37th'},
             '38': {'New Jersey 38', 'New Jersey Route 38', 'Route 38'},
             '39th': {'North 39th'},
             '40': {'1140 US Highway 40'},
             '4080': {'4080'},
             '41st': {'South 41st'},
             '43rd': {'North 43rd'},
             '446-1234': {'1 Brookline Boulevard PA 19083(610) 446-1234'},
             '452': {'M

In [10]:
# 2.57 min 1/5 data, full data likely over 12 min. 2nd version took about 11 min for 1/5 data. 
filter_words(sample_file_k5)

In [11]:
audit(sample_file_k5)

defaultdict(set,
            {'206': {'US 70 and US 206'},
             '33': {'Route 33'},
             '37th': {'North 37th'},
             '446-1234': {'1 Brookline Boulevard PA 19083(610) 446-1234'},
             '5': {'West Girard Avenue, 5'},
             '70': {'NJ 70', 'US 70'},
             '73': {'New Jersey 73'},
             '80': {'North Lewis Road Unit #80'},
             'Moore': {'Cecil B. Moore'},
             'NJ-73': {'NJ-73'},
             'PA': {'East Lincoln Highway PA'},
             'Vine': {'12th and Vine'}})

### Approximate (Fuzzy) String Matching

In [59]:
from fuzzywuzzy import fuzz, process
import time, sys

def fix_street_names(osmfile):
    start_time = time.time()
    reload(sys)
    sys.setdefaultencoding('utf-8')
    soup = BeautifulSoup(open(osmfile, "r+b"), "xml")
    
    street_tags = soup.find_all("tag", attrs={"k": "addr:street"})
    
    with open('./philly_street_full_names_canon.csv', 'r') as streets_canon:
        reader = csv.reader(streets_canon)
        possible_streets = list(reader)[0]
    
    no_fix = ['Mallon Avenue']
    
    # Only run for those records yielded from the audit function
    for tag in street_tags:
        m = street_type_re.search(tag['v'])
        street_end = m.group()
        intersect = intersection.search(tag['v'])

        if street_end not in expected and not intersect and tag['v'] not in no_fix: 
            st_name = process.extractOne(tag['v'], possible_streets)
            if st_name[1] >= 90:
                tag['v'] = st_name[0]

    with open(osmfile, "w") as f:
        f.write(soup.prettify())
    print("--- {}min ---".format((time.time() - start_time)/60))


In [60]:
# 25-28 min runtime
fix_street_names(clean_full_osm)

In [61]:
# Intersections aren't being fixed as they should be. Does it make a difference if they have 'Streets' at the end? 
audit(clean_full_osm)

defaultdict(set,
            {'1': {'Route 1'},
             '206': {'US 206', 'US 70 and US 206'},
             '33': {'Route 33'},
             '38': {'New Jersey 38', 'New Jersey Route 38', 'Route 38'},
             '40': {'1140 US Highway 40', 'Rt 40'},
             '4080': {'4080'},
             '70': {'NJ 70', 'US 70'},
             '73': {'NJ 73', 'New Jersey 73', 'North Route 73', 'Route 73'},
             'Brown': {'North 37th and Brown'},
             'Master': {'15th and Master'},
             'NJ-73': {'NJ-73'},
             'Nixon': {'Shawmont and Nixon'},
             'Reno': {'North 50th Street and Reno'},
             'Stiles': {'16th and Stiles'},
             'Thompson': {'Sletcher and Thompson'},
             'Vine': {'12th and Vine'},
             'susquahana': {'thompson and susquahana'}})

In [13]:
# First version took 7 hours. Whew, only 8 min this time, after narrowing what fuzzy string match does. Stil, full file might take 
# 40 min or more. Now 6 min for 20% of file. w00t!
fix_street_names(sample_file_k5)

In [14]:
# Highways and intersections aren't consistent - are intersections valid addr:street attribute values? What format should state and 
# national highways follow?
audit(sample_file_k5)

defaultdict(set,
            {'206': {'US 70 and US 206'},
             '33': {'Route 33'},
             '70': {'NJ 70', 'US 70'},
             '73': {'New Jersey 73'},
             'NJ-73': {'NJ-73'},
             'Vine': {'12th and Vine'}})

## Write Cleaned OSM XML to CSV

In [65]:
import csv
import codecs
import re
import xml.etree.cElementTree as ET
import schema

OSM_PATH = "full_osm_clean.osm"

NODES_PATH = "./db_csvs/nodes.csv"
NODE_TAGS_PATH = "./db_csvs/node_tags.csv"
WAYS_PATH = "./db_csvs/ways.csv"
WAY_NODES_PATH = "./db_csvs/way_nodes.csv"
WAY_TAGS_PATH = "./db_csvs/way_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  

    for e in element.iter("tag"):
        id = element.get('id')
        key = e.get('k')
        value = e.get('v')
        low = LOWER_COLON.search(key)
        prob = PROBLEMCHARS.search(key)
        if low and not prob:
            kt = key.split(':', 1)
            tags.append({"id": id, "key": kt[1], "value": value, "type": kt[0]})
        if not low and not prob:
            tags.append({"id": id, "key": key, "value": value, "type": 'regular'})
        if prob:
            continue
        
    if element.tag == 'node':
        for idx, val in enumerate(NODE_FIELDS):
            k = NODE_FIELDS[idx] 
            val = element.get(val)
            node_attribs.update({k: val})    
        if node_attribs:
            return {'node': node_attribs, 'node_tags': tags}
        else:
            return None   
    
    elif element.tag == 'way':
        for idx, val in enumerate(WAY_FIELDS):
            k = WAY_FIELDS[idx]
            val = element.get(val)
            way_attribs.update({k: val})
        i = 0
        for w in element.iter('nd'):
            id = element.get('id')
            n_id = w.get('ref')
            way_nodes.append({"id": id, "node_id": n_id, "position": i})
            i += 1
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}
    
#     print node_attribs
#     print way_attribs
#     print way_nodes
#     print tags

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()
            
def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_strings = (
            "{0}: {1}".format(k, v if isinstance(v, str) else ", ".join(v))
            for k, v in errors.iteritems()
        )
        raise cerberus.ValidationError(
            message_string.format(field, "\n".join(error_strings))
        )


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

In [110]:
for element in get_element(OSM_PATH, tags=('node', 'way')):
    x = shape_element(element)
    print x

In [66]:
# 31 min with validator on 1/5 file
import cerberus
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""
    start_time = time.time()
    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])
    print("--- {}min ---".format((time.time() - start_time)/60))

# if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
#     process_map(sample_file_k5, validate=False)


In [67]:
process_map(OSM_PATH, validate=False)

### Create SQL Database and Specify Schema

In [68]:
from sqlalchemy import create_engine, ForeignKey, Column, Integer, Float, Date, String
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlite3 import dbapi2 as sqlite

engine = create_engine('sqlite:///./db_csvs/philly_test_sql.db', module=sqlite)

Base = declarative_base()

class Node(Base):
    __tablename__ = 'Nodes'
    id = Column(Integer, primary_key=True, nullable=False) 
    lat = Column(Float)
    lon = Column(Float)
    user = Column(String)
    uid = Column(Integer)
    version = Column(String)
    changeset = Column(Integer)
    timestamp = Column(String)
    
class Node_Tag(Base):
    __tablename__ = 'Node_Tags'
    id = Column(Integer, ForeignKey("Nodes.id"), nullable=False) 
    key = Column(String)
    value = Column(String)
    type = Column(String)
    idx = Column(Integer, primary_key=True, index=True, unique=True, nullable=True)

class Way(Base):
    __tablename__ = 'Ways'
    id = Column(Integer, primary_key=True, nullable=False) 
    user = Column(String)
    uid = Column(Integer)
    version = Column(String)
    changeset = Column(Integer)
    timestamp = Column(String)

class Way_Node(Base):
    __tablename__ = 'Way_Nodes'
    id = Column(Integer, ForeignKey("Ways.id"), nullable=False) 
    node_id = Column(Integer)
    position = Column(Integer, nullable=True)
    idx = Column(Integer, primary_key=True, index=True, unique=True, autoincrement=True)

class Way_Tag(Base):
    __tablename__ = 'Way_Tags'
    id = Column(Integer, ForeignKey("Ways.id"), nullable=False) 
    key = Column(String)
    value = Column(String)
    type = Column(String)
    idx = Column(Integer, primary_key=True, index=True, unique=True, nullable=False)


Base.metadata.create_all(engine)

### Load Data from CSV Files and Write Records to SQL DB 

In [69]:
import csv
from time import time
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

def UnicodeDictReader(utf8_data, **kwargs):
    csv_reader = csv.DictReader(utf8_data, **kwargs)
    for row in csv_reader:
        yield {key: unicode(value, 'utf-8') for key, value in row.iteritems()}

def load_dat(filename):    
    table_stem = filename.split('/')[2]
    table_name = '_'.join([x.capitalize() for x in table_stem.split('_')]).strip('.csv')+'s'
    
    engine = create_engine('sqlite:///./db_csvs/philly_test_sql.db')
    session = sessionmaker()
    session.configure(bind=engine)
    s = session()

    with open(filename, 'rb') as f:
        fiel = csv.DictReader(f)
        fields = list(fiel)[0].keys()
        n = len(fields)
        if n == 8:
            with open(filename, 'rb') as g:
                fr = UnicodeDictReader(g)
                dat = [{'id': i['id'], 'lat': i['lat'], 'lon': i['lat'], 'user': i['user'], 'uid': i['uid'], 
                        'version': i['version'], 'changeset': i['changeset'], 'timestamp': i['timestamp']} for i in fr]      
            for row in dat:
                record = Node(**row)
                s.add(record)
            s.commit()
            s.close()    
        if n == 6:
            with open(filename, 'rb') as g:
                fr = UnicodeDictReader(g)
                dat = [{'id': i['id'], 'user': i['user'], 'uid': i['uid'], 'version': i['version'], 'changeset': i['changeset'],
                        'timestamp': i['timestamp']} for i in fr]
            for row in dat:
                record = Way(**row)
                s.add(record)
            s.commit()
            s.close()
        if n == 4:
            with open(filename, 'rb') as g:
                fr = UnicodeDictReader(g)
                dat = [{'id': i['id'], 'key': i['key'], 'value': i['value'], 'type': i['type']} for i in fr]       
            for row in dat:
                if table_name == 'Node_Tags':
                    record = Node_Tag(**row)
                    s.add(record)
                if table_name == 'Way_Tags':
                    record = Way_Tag(**row)
                    s.add(record)
            s.commit()
            s.close
        if n == 3:
            with open(filename, 'rb') as g:
                fr = UnicodeDictReader(g)
                dat = [{'id': i['id'], 'node_id': i['node_id'], 'position': i['position']} for i in fr]
            for row in dat:
                record = Way_Node(**row)
                s.add(record)
            s.commit()
            s.close()        

In [70]:
import fnmatch
import os

files = fnmatch.filter(os.listdir('./db_csvs'), '*.csv')

for f in files:
    fname = './db_csvs/{}'.format(f)
    load_dat(fname) 

#     t ='{}'.format('_'.join([x.capitalize() for x in f.split('_')]).strip('.csv') + 's')
#     print t
#     table_name = '_'.join([x.capitalize() for x in f.split('_')]).strip('.csv') + 's'
#     print table_name
#     print fname

# files


In [75]:
import sqlite3
conn = sqlite3.connect('./db_csvs/philly_test_sql.db')

c = conn.cursor()

x = c.execute("""SELECT * FROM sqlite_master WHERE type='table';""")
y = c.execute("""SELECT count(*) FROM Nodes;""")

print x.fetchall()
# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()